# Model

We suppose that the default model of the each member is given by:

\begin{equation*}
\mathbb{P}(\tau_i \leq t) = 1 - \exp \left( - \sum_{i=0}^{k} \lambda_i (T_{i+1}-T_{i}) - \lambda_k (t-T_k) \right)
\end{equation*}

for $t \in [T_k, T_{k+1}[$, with the convention that $T_0 = 0$.

# Load matlab file

In [1]:
import scipy.io

mat = scipy.io.loadmat('Calibrated_Common_Shock_with_lambda_ind.mat')
print mat.keys()

print
print mat['__header__']
print mat['__version__']
print mat['__globals__']

['Calibrated_Common_Choc', '__version__', '__header__', '__globals__']

MATLAB 5.0 MAT-file, Platform: MACI64, Created on: Tue Apr  7 14:26:40 2015
1.0
[]


In [2]:
%matplotlib inline
import matplotlib.pyplot as plt

In [3]:
import numpy as np

calib_comm_shocks = mat['Calibrated_Common_Choc'][0, 0]
calib_comm_shocks.dtype

dtype([('Names', 'O'), ('Sectors', 'O'), ('Currencies', 'O'), ('CDS_Pillars', 'O'), ('Today', 'O'), ('coupDate', 'O'), ('CDS_Spreads', 'O'), ('CDO_attach', 'O'), ('CDO_detach', 'O'), ('CDO_Spreads', 'O'), ('CDO_Maturity', 'O'), ('Eta', 'O'), ('CDS_Pillars_in_years', 'O'), ('Common_Choc_Pillars', 'O'), ('NbNames', 'O'), ('Recovery', 'O'), ('IR', 'O'), ('Rank_group', 'O'), ('Group_size', 'O'), ('Lambda_group_optimal', 'O'), ('Lambda_ind', 'O'), ('Group_Composition', 'O')])

In [4]:
pillars = [3, 5]

## Group structure and intensities

### Group structure

In [5]:
raw_groups = calib_comm_shocks["Group_Composition"]
print raw_groups.dtype.fields

{'I_disjnt_3': (dtype('O'), 16), 'I_disjnt_2': (dtype('O'), 8), 'I_disjnt_1': (dtype('O'), 0), 'I_disjnt_5': (dtype('O'), 32), 'I_disjnt_4': (dtype('O'), 24)}


In [6]:
import re

group_composition = {int(re.findall(r'\d+', x)[0]): frozenset(raw_groups[x][0][0][0][0][0].flatten()) 
                     for x in raw_groups.dtype.fields}

In [7]:
for g in group_composition:
    print g, group_composition[g], len(group_composition[g])

1 frozenset([99, 100, 39, 105, 76, 27]) 6
2 frozenset([32, 65, 82, 71, 72, 77, 79, 112, 18, 84, 121, 125, 30]) 13
3 frozenset([67, 104, 108, 51, 56, 123]) 6
4 frozenset([1, 4, 5, 6, 10, 14, 25, 26, 28, 29, 31, 34, 37, 38, 44, 46, 48, 53, 57, 58, 62, 73, 74, 81, 83, 87, 88, 90, 91, 101, 113, 114, 115, 117, 119, 120]) 36
5 frozenset([2, 3, 7, 8, 9, 11, 12, 13, 15, 16, 17, 19, 20, 21, 22, 23, 24, 33, 35, 36, 40, 41, 42, 43, 45, 47, 49, 50, 52, 54, 55, 59, 60, 61, 63, 64, 66, 68, 69, 70, 75, 78, 80, 85, 86, 89, 92, 93, 94, 95, 96, 97, 98, 102, 103, 106, 107, 109, 110, 111, 116, 118, 122, 124]) 64


In [8]:
groups_intensities = {}

raw_group_intensities = calib_comm_shocks['Lambda_group_optimal']
for ind, pill in enumerate(pillars):
    raw_lambdas_pills = raw_group_intensities[:, ind]
    groups_intensities[pill] = {i+1: c for i, c in enumerate(raw_lambdas_pills)}

In [9]:
print groups_intensities[3]
print groups_intensities[5]

{1: 0.058000000000000003, 2: 0.0053744999999999999, 3: 0.00061450000000000003, 4: 0.00020165, 5: 0.0016207000000000001}
{1: 0.015925000000000002, 2: 0.0050889000000000004, 3: 0.0086692000000000002, 4: 9.3372e-05, 5: 0.0076647}


## Idiosyncratic intensities

In [10]:
raw_idio = calib_comm_shocks['Lambda_ind']

I_5 = group_composition[5]

idios = []
nb_obligors = raw_idio.shape[0]

for i, idio in enumerate(raw_idio):
    id_ = i+1
    
    if id_ in I_5:
        continue
    
    lambda_3y = idio[0]
    lambda_5y = idio[1]
    
    idios.append({"id": frozenset([i]), "pillars": [3, 5], "hazard_rates": [lambda_3y, lambda_5y]})

In [11]:
print "Carefull of these obligors:\n--------------------------"
for d in idios:
    if min(d['hazard_rates']) < 0:
        print "ID = %s | l_3y = %s | l_5y = %s"%(d['id'], d['hazard_rates'][0], d['hazard_rates'][1])

Carefull of these obligors:
--------------------------
ID = frozenset([4]) | l_3y = 0.0085808902955 | l_5y = -4.31167802529e-08


## Final intensities

In [12]:
groups_save = []

ids_ = frozenset()
for group_nb, ids in group_composition.iteritems():
    ids_ = ids_.union(ids)
    ids__ = [id_-1 for id_ in ids_]
    
    print "Length for %i = %i"%(group_nb, len(ids_))
    d = {"group_id": group_nb, "ids": frozenset(ids__), "pillars": [3, 5],
         "hazard_rates": [groups_intensities[3][group_nb], groups_intensities[5][group_nb]]}
    
    groups_save.append(d)

Length for 1 = 6
Length for 2 = 19
Length for 3 = 25
Length for 4 = 61
Length for 5 = 125


In [13]:
idio_save = idios

In [14]:
to_save = {
    "idio": idio_save,
    "groups": groups_save
}

In [15]:
import dill

In [16]:
with open('intensities_no_I5_new.pkl', 'wb') as f:
    dill.dump(to_save, f)

In [17]:
with open('intensities_no_I5_new.pkl', 'rb') as f:
    test = dill.load(f)

In [18]:
for i in test['idio']:
    print "-----------------------------------"
    print "Bank %i" %list(i['id'])[0]
    print "-----------------------------------"
    print "Pillars: %s\nIntensities: %s" %(i['pillars'], i['hazard_rates'])
    print

-----------------------------------
Bank 0
-----------------------------------
Pillars: [3, 5]
Intensities: [0.0033853684009316522, 0.0048081762873048344]

-----------------------------------
Bank 3
-----------------------------------
Pillars: [3, 5]
Intensities: [0.0035647268350895425, 0.0049729999603603187]

-----------------------------------
Bank 4
-----------------------------------
Pillars: [3, 5]
Intensities: [0.0085808902954963729, -4.3116780252920595e-08]

-----------------------------------
Bank 5
-----------------------------------
Pillars: [3, 5]
Intensities: [0.0071488558192109308, 0.0021660883207161351]

-----------------------------------
Bank 9
-----------------------------------
Pillars: [3, 5]
Intensities: [0.0053575143772381798, 0.0031357382970320863]

-----------------------------------
Bank 13
-----------------------------------
Pillars: [3, 5]
Intensities: [0.00679070303904733, 0.011875018141229196]

-----------------------------------
Bank 17
--------------------

In [19]:
for g in test['groups']:
    print "-----------------------------------"
    print "Group %i" %g['group_id']
    print "-----------------------------------"
    print "Pillars: %s\nIntensities: %s" %(g['pillars'], g['hazard_rates'])
    print "Composition: %s" %g['ids']
    print

-----------------------------------
Group 1
-----------------------------------
Pillars: [3, 5]
Intensities: [0.058000000000000003, 0.015925000000000002]
Composition: frozenset([98, 99, 38, 104, 75, 26])

-----------------------------------
Group 2
-----------------------------------
Pillars: [3, 5]
Intensities: [0.0053744999999999999, 0.0050889000000000004]
Composition: frozenset([64, 98, 99, 38, 70, 71, 104, 75, 76, 78, 111, 17, 83, 120, 26, 31, 124, 29, 81])

-----------------------------------
Group 3
-----------------------------------
Pillars: [3, 5]
Intensities: [0.00061450000000000003, 0.0086692000000000002]
Composition: frozenset([17, 26, 29, 31, 38, 50, 55, 64, 66, 70, 71, 75, 76, 78, 81, 83, 98, 99, 103, 104, 107, 111, 120, 122, 124])

-----------------------------------
Group 4
-----------------------------------
Pillars: [3, 5]
Intensities: [0.00020165, 9.3372e-05]
Composition: frozenset([0, 3, 4, 5, 9, 13, 17, 24, 25, 26, 27, 28, 29, 30, 31, 33, 36, 37, 38, 43, 45, 47, 50